# Training cycle debugging

Basically run code copied from train_seq2seq.py in this notebook to catch bugs

In [2]:
import os
os.environ['ALFRED_ROOT'] = '/root/data/home/hoyeung/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

# from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
import torch
import pprint
import json
from data.preprocess import Dataset
from importlib import import_module, reload
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from models.utils.helper_utils import optimizer_to

# Parser

In [3]:
# These are the default flags present in train_seq2seq.py

parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

# settings
args.seed = 123
args.data = '/root/data_alfred/json_feat_2.1.0'
args.splits = '/root/data_alfred/splits/may17.json'
args.preprocess = False #!
args.pp_folder = 'pp'
args.monitor_train_every = 10
args.save_every_epoch = False #!
args.model = 'seq2seq_per_subgoal'
args.gpu = True
args.dout = 'exp/model:seq2seq_per_subgoal'
args.resume = False #!

# hyper parameters
args.batch = 8
args.epoch = 20
args.lr = 1e-4
args.decay_epoch = 10
args.dhid = 512
args.dframe = 2500
args.demb = 100
args.pframe = 300
args.mask_loss_wt = 1.
args.action_loss_wt = 1.
args.subgoal_aux_loss_wt = 0.
args.pm_aux_loss_wt = 0.

# architecture ablations
args.maxpool_over_object_states = False
args.aux_loss_over_object_states = False

# dropouts
args.zero_goal = False #!
args.zero_instr = False #!
args.act_dropout = 0.
args.lang_dropout = 0.
args.input_dropout = 0.
args.vis_dropout = 0.3
args.hstate_dropout = 0.3
args.attn_dropout = 0.
args.actor_dropout = 0.
args.word_dropout = 0.

# other settings
args.train_teacher_forcing = False #!
args.train_student_forcing_prob = 0.1
args.temp_no_history = False #!

# debugging
args.fast_epoch = False #!
args.dataset_fraction = 0

In [4]:
# overwrite the default flags

args.preprocess = False # Turn this to True if running for the first time

args.model = 'seq2seq_per_subgoal'  # found under models/model/ directory
args.dout = '/root/data_alfred/exp/model:seq2seq_per_subgoal_fast_epoch'

args.train_teacher_forcing = True
args.gpu = False

# light setup for debugging
args.fast_epoch = True # Turn this to False if running for the first time to preprocess data properly
args.epoch = 20

In [5]:
args.maxpool_over_object_states = True
args.aux_loss_over_object_states = True

In [6]:
torch.manual_seed(args.seed)

# Setup and load data splits

In [7]:
# make output dir
if not os.path.isdir(args.dout):
    os.makedirs(args.dout)

print('args.out ', args.dout)

args.out  /root/data_alfred/exp/model:seq2seq_per_subgoal_fast_epoch


In [8]:
# load train/valid/tests splits
with open(args.splits) as f:
    splits = json.load(f)
    pprint.pprint({k: len(v) for k, v in splits.items()})

{'tests_seen': 1533,
 'tests_unseen': 1529,
 'train': 20806,
 'train_sanity': 246,
 'train_sanity_v1': 246,
 'valid_seen': 814,
 'valid_seen_v1': 249,
 'valid_unseen': 818,
 'valid_unseen_v1': 254}


In [9]:
# preprocess and save -- only need to preprocess once
if args.preprocess:
    print("\nPreprocessing dataset and saving to %s folders ... This will take a while. Do this once as required." % args.pp_folder)
    dataset = Dataset(args, None)
    dataset.preprocess_splits(splits)
    vocab = torch.load(os.path.join(args.dout, "%s.vocab" % args.pp_folder))
else:
    vocab = torch.load(os.path.join(args.data, "%s.vocab" % args.pp_folder))
    
print(vocab)

{'action_high': Vocab(93), 'word': Vocab(2360), 'action_low': Vocab(15)}


In [10]:
object_vocab = torch.load(os.path.join(args.data, '%s.vocab' % 'objects'))

# Model and Training

In [11]:
# load the model architecture
args.gpu = False
args.resume = None # os.path.join(args.dout, 'best_seen.pth')

M = import_module('model.{}'.format(args.model))
if args.resume:
    print("Loading: " + args.resume)
    model, optimizer, start_epoch = M.Module.load(args.resume)
    print("Restarting at epoch {}/{}".format(start_epoch, args.epoch-1))
    if start_epoch >= args.epoch:
        print('Checkpoint already finished {}/{} epochs.'.format(start_epoch, args.epoch))
        sys.exit(0)
else:
    model = M.Module(args, vocab, object_vocab)
    optimizer = None
    start_epoch = 0

if args.gpu:
    model = model.to(torch.device('cuda'))
    if not optimizer is None:
        optimizer_to(optimizer, torch.device('cuda'))

In [13]:
# examine model layers
model

Module(
  (emb_word): Embedding(2360, 100)
  (emb_action_low): Embedding(15, 100)
  (emb_object): Embedding(109, 512, padding_idx=0)
  (enc): ActionFrameAttnEncoderPerSubgoal(
    (emb): Embedding(15, 100)
    (vis_dropout): Dropout(p=0.3, inplace=False)
    (act_dropout): Dropout(p=0.0, inplace=True)
    (vis_encoder): ResnetVisualEncoder(
      (conv1): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
      (conv2): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
      (fc): Linear(in_features=3136, out_features=2500, bias=True)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (enc_att): SelfAttn(
      (scorer): Linear(in_features=1024, out_features=1, bias=True)
    )
    (obj_emb): Embedding(109, 512, padding_idx=0)
    (encoder): LSTM(3624, 512, batch_first=True, bidirectional=True)
  )
  (dec): LanguageDecoder(
    (emb): Embed

In [14]:
# main training loop -- debug here if breakpoints were inserted
model.run_train(splits, optimizer=optimizer, start_epoch=start_epoch)

batch:   0%|          | 0/2 [00:00<?, ?it/s]

Saving to: /root/data_alfred/exp/model:seq2seq_per_subgoal_fast_epoch


/root/data/home/hoyeung/alfred/models/model/seq2seq_per_subgoal.py:253: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  empty_tensor = torch.ones(torch.tensor(v[0][0][0]).unsqueeze(0).shape, device=device, dtype=torch.float if ('frames' in k) else torch.long) * self.pad
/root/data/home/hoyeung/alfred/models/model/seq2seq_per_subgoal.py:264: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seqs.append(torch.tensor(v[subgoal_i][batch_i], device=device, dtype=torch.float if ('frames' in k) else torch.long))
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functio

> /root/data/home/hoyeung/alfred/models/model/seq2seq.py(53)log_metrics()
-> self.summary_writer.add_scalar('{}/{}_{}_{}'.format(split_name, prefix, k, suffix), metrics[k], ix)
(Pdb) metrics[k]
5.020378108314667e-232
(Pdb) c



batch: 100%|██████████| 2/2 [00:19<00:00,  9.55s/it]


...

{'epoch': 0,
 'train_sanity': {'BLEU': 5.020378108314667e-232,
                  'perplexity': 191.19113159179688,
                  'total_loss': 12.033584594726562},
 'valid_seen': {'BLEU': 3.182632276440731e-157,
                'perplexity': 187.1081314086914,
                'total_loss': 11.880884170532227},
 'valid_unseen': {'BLEU': 2.0138941114446933e-157,
                  'perplexity': 185.96180725097656,
                  'total_loss': 12.561946392059326}}
epoch_time                    411.009                                 
compute_metrics_validation_sets389.498                                 
forward_batch_train_with_iterate20.178                                  
forward_batch_train           19.882                                  
torch_save_last               0.282                                   
torch_save_valid_seen         0.279                                   
torch_save_valid_unseen       0.277                                   
torch_save_train_sanity     


batch: 100%|██████████| 2/2 [00:08<00:00,  4.48s/it]

batch: 100%|██████████| 2/2 [00:02<00:00,  1.05s/it]

batch: 100%|██████████| 2/2 [00:02<00:00,  1.38s/it]

batch: 100%|██████████| 2/2 [00:02<00:00,  1.30s/it]

batch: 100%|██████████| 2/2 [00:14<00:00,  7.33s/it]


> /root/data/home/hoyeung/alfred/models/model/seq2seq.py(53)log_metrics()
-> self.summary_writer.add_scalar('{}/{}_{}_{}'.format(split_name, prefix, k, suffix), metrics[k], ix)
(Pdb) q


epoch:   5%|▌         | 1/20 [07:35<2:24:05, 455.04s/it]


BdbQuit: 

In [ ]:
feat['object_visibility'][subgoal_i][torch.arange(batch_size),action_low_seq_lengths,:]

In [ ]:
feat['object_visibility'][subgoal_i][torch.arange(feat['action_low'][0].shape[0]),action_low_seq_lengths,:]

In [ ]:
11, 17, 42

In [ ]:
epoch_time                    110.233                                 
compute_metrics_validation_sets87.779                                  
forward_batch_train_with_iterate14.776                                  
forward_batch_train           14.518                                  
torch_save_train_sanity       3.805                                   
torch_save_valid_unseen       2.162                                   
make_debug_train              0.928                                   
torch_save_last               0.341                                   
torch_save_valid_seen         0.296                                   
iterate_featurize             0.244                                   
iterate_load_task_json        0.161                                   
featurize_input_resnet_features0.115                                   
featurize_tensorization_and_padding0.113                                   
make_debug_valid_seen         0.099                                   
featurize_torch_load_time     0.06                                    
make_debug_valid_unseen       0.045                                   
setup_time                    0.001                                   
featurize_outputs             0.001                                   
featurize_input_action_low    0.0                                     
compute_metrics_train         0.0  